In [16]:
from requests import get
import pandas as pd
from numpy import NaN
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [54]:
class StatsNBA():
    def __init__(self, team, start_year, end_year = 2023):
        self.team = team
        self.start_year = int(start_year)
        self.end_year = int(end_year)
        self.columns = ['G','Date','Start (ET)','#','##','Out Home','Opponent', 'W\L', 'OT','Tm','Opp','W','L','Streak','Notes']

    def get_HTML(self, year):
        url = f'https://www.basketball-reference.com/teams/{self.team}/{year}_games.html'
        r = get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        return soup
    

    def get_stats(self):
        

        # Criando dataframes
        df_data = pd.DataFrame(columns=self.columns)
        df_StatsNBA = pd.DataFrame(columns=self.columns) 

        rows = []
        count_index = 1
        count_year = self.start_year

        while count_year <= self.end_year:

            html = self.get_HTML(count_year)
            games = html.find_all('tr')
        
            try:
                for game in games[1:]:

                    # Verificando se a coluna [G] é um numero
                    if game.findAll('th')[0].get_text().isnumeric():
                            
                            # Armazenando o numero do jogo e fazendo append na lista de linhas 
                            numGame = game.findAll('th')[0].get_text()
                            rows.append(numGame)

                            for infoJogo in game.findAll('td'):
                                rows.append(infoJogo.get_text())

                            # adicionando os dados no datafreme
                            df_data[self.columns] = [rows]
                            df_StatsNBA = pd.concat([df_StatsNBA, df_data])
                            rows.clear()
            except:
                rows.clear()
                for game in games[1:]:


                    # Verificando se a coluna [G] é um numero
                    if game.findAll('th')[0].get_text().isnumeric():
                            
                            # Armazenando o numero do jogo e fazendo append na lista de linhas 
                            numGame = game.findAll('th')[0].get_text()
                            rows.append(numGame)

                            # for para percorrer a tabela que contem as estatisticas e armazenando na lista de linhas    
                            for infoJogo in game.findAll('td'):
                            
                                if count_index == 2:
                                    
                                    rows.append(NaN)
                                    rows.append(infoJogo.get_text())
                                        
                                    count_index+=1
                            
                                else:
                                    rows.append(infoJogo.get_text())
                                    count_index+=1

                            count_index = 1

                            # Adicionando as Linhas no DataFrame
                            df_data[self.columns] = [rows]
                            df_StatsNBA = pd.concat([df_StatsNBA, df_data])
                            rows.clear()
            
            count_year += 1
            sleep(randint(1, 5))
        
        df_StatsNBA.drop(['##', '#', 'Notes'], axis=1, inplace=True)
        
        return df_StatsNBA

In [58]:
LAL = StatsNBA('LAL', 2020, 2020)

In [59]:
LAL.get_stats()

,G,Date,Start (ET),Out Home,Opponent,W\L,OT,Tm,Opp,W,L,Streak
0,1,"Tue, Oct 22, 2019",10:30p,@,Los Angeles Clippers,L,,102,112,0,1,L 1
0,2,"Fri, Oct 25, 2019",10:30p,,Utah Jazz,W,,95,86,1,1,W 1
0,3,"Sun, Oct 27, 2019",9:30p,,Charlotte Hornets,W,,120,101,2,1,W 2
0,4,"Tue, Oct 29, 2019",10:30p,,Memphis Grizzlies,W,,120,91,3,1,W 3
0,5,"Fri, Nov 1, 2019",9:30p,@,Dallas Mavericks,W,OT,119,110,4,1,W 4
...,...,...,...,...,...,...,...,...,...,...,...,...
0,17,"Fri, Oct 2, 2020",9:00p,,Miami Heat,W,,124,114,14,3,W 4
0,18,"Sun, Oct 4, 2020",7:30p,@,Miami Heat,L,,104,115,14,4,L 1
0,19,"Tue, Oct 6, 2020",9:00p,@,Miami Heat,W,,102,96,15,4,W 1
0,20,"Fri, Oct 9, 2020",9:00p,,Miami Heat,L,,108,111,15,5,L 1
